In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch as t
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch.nn.functional as F
import seaborn as sns
import torch


In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  
        out = F.relu(out)
        return out
    
    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Sequential(nn.Linear(256, num_classes),
                                )
    
    def _make_layer(self, block, out_channels, blocks, stride):
        layers = []

        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = (self.fc1(out))
        return out


In [7]:
import os,cv2 as cv



device = torch.device("mps" if torch.mps.is_available() else "cpu")
model = ResNet(ResidualBlock,[2,2,2],10).to(device)
model.load_state_dict(t.load("model.pth", map_location=device))
model.eval()

class_labels = ['aeroplane','automobile','bird','catt','deer','dog','frog','horse','ship','truck']

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

def test(x):
    x = cv.imread(x)
    lab = ['aeroplane','automobile','bird','catt','deer','dog','frog','horse','ship','truck']
    
    with t.no_grad():
        
        
            
        transform = transforms.Compose([
        transforms.ToPILImage(),                   
        transforms.Grayscale(num_output_channels=1), 
        transforms.Resize((32, 32)),                  
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]) 
    ])

        x = transform(x)
        x=x.unsqueeze(0)
        x = x.to(device)

        ypred = model(x)
        _, predicted = t.max(ypred.data, 1)
        
        
        return (lab[predicted])

In [10]:
test("/Users/bhara-zstch1566/CNN/Project/CIFRA 10/deer.jpeg")

'deer'